In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit as ss

from DataPreprocessing.my_utils import get_fairness, get_score, vis
from Maximus_optimized_non_dominated import Multi_Fair as maximus

/home/philip/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/philip/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
def train_classifier1(
    X_train, X_test, y_train, y_test, sa_index, p_Group, base_learners, preference
):

    classifier = maximus(
        n_estimators=base_learners,
        saIndex=sa_index,
        saValue=p_Group,
        preference=preference,
    )

    classifier.fit(X_train, y_train)

    y_pred_probs = classifier.predict_proba(X_test)[:, 1]
    y_pred_labels = classifier.predict(X_test)
    f = classifier.feature_importances_
    # return classifier.conf_scores, classifier.get_weights_over_iterations(), classifier.get_initial_weights()
    return y_pred_probs, y_pred_labels, classifier

In [17]:
def train(X, y):
    in_ts, pred1, fx = [], [], []
    sss = ss(n_splits=5, test_size=0.4)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        in_ts.append(test_index)
        pb1, pd1, f1 = train_classifier1(
            X_train,
            X_test,
            y_train,
            y_test,
            sa_index,
            p_Group,
            499,
            [
                [0, 0.5, 0.5],
                [0.2, 0.4, 0.4],
                [0.4, 0.2, 0.4],
                [0.4, 0.4, 0.2],
                [0.5, 0.5, 0],
                [0.5, 0, 0.5],
                [1, 0, 0],
                [0, 1, 0],
                [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
            ],
        )
        pred1.append(pd1)
        fx.append(f1)
        # print(f1.theta - 1, " : ", f1.ob[f1.theta - 1])
    return in_ts, pred1, fx

In [30]:
results, performance, Hx = [], [], []
for dt in ["Adult"]:
    if dt == "Adult":
        from DataPreprocessing.load_adult import load_adult

        X, y, sa_index, p_Group, x_control, F = load_adult()
        how_many = 1000
        X = X[:how_many, :]
        y = y[:how_many]
        x_control["race"] = x_control["race"][:how_many]
        x_control["sex"] = x_control["sex"][:how_many]
        # v='Adult_2_sensi_Mari_Sex'
        saf = sa_index[1]
    elif dt == "Bank":
        from DataPreprocessing.load_bank import load_bank

        X, y, sa_index, p_Group, x_control, F = load_bank()
        saf = sa_index[0]
        print(saf)
    elif dt == "Credit":
        from DataPreprocessing.load_credit import load_credit

        X, y, sa_index, p_Group, x_control, F = load_credit()
        saf = sa_index[0]
    elif dt == "Compas":
        from DataPreprocessing.load_compas_data import load_compas

        X, y, sa, p_G, x_control, F = load_compas()
        sa_index = [sa[-1], sa[0]]
        p_Group = [p_G[-1], p_G[0]]
    sensitives = [F[v] for v in sa_index]
    in_ts, pred1, f1 = train(X, y)
    results.append(list(get_fairness(sa_index, p_Group, in_ts, pred1, X, y).values()))
    performance.append(get_score(pred1, in_ts, X, y))
    Hx.append(f1)

45175
Features we will be using for classification are: ["workclass_' Federal-gov'", "workclass_' Local-gov'", "workclass_' Private'", "workclass_' Self-emp-inc'", "workclass_' Self-emp-not-inc'", "workclass_' State-gov'", "workclass_' Without-pay'", "education_' 10th'", "education_' 11th'", "education_' 12th'", "education_' 1st-4th'", "education_' 5th-6th'", "education_' 7th-8th'", "education_' 9th'", "education_' Assoc-acdm'", "education_' Assoc-voc'", "education_' Bachelors'", "education_' Doctorate'", "education_' HS-grad'", "education_' Masters'", "education_' Preschool'", "education_' Prof-school'", "education_' Some-college'", "Maritial-status_' Divorced'", "Maritial-status_' Married-AF-spouse'", "Maritial-status_' Married-civ-spouse'", "Maritial-status_' Married-spouse-absent'", "Maritial-status_' Never-married'", "Maritial-status_' Separated'", "Maritial-status_' Widowed'", "occupation_' Adm-clerical'", "occupation_' Armed-Forces'", "occupation_' Craft-repair'", "occupation_' 

In [31]:
Hx[0][0].preference

[[0, 0.5, 0.5],
 [0.2, 0.4, 0.4],
 [0.4, 0.2, 0.4],
 [0.4, 0.4, 0.2],
 [0.5, 0.5, 0],
 [0.5, 0, 0.5],
 [1, 0, 0],
 [0, 1, 0],
 [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]

In [32]:
import numpy as np

from selection import PreferenceSurvival

In [33]:
preference_vectors = np.array(Hx[0][0].preference)#.reshape(1, -1)
# Add more for testing
#preference_vectors = np.row_stack(
#    [[0, 0.8, 0.2], [1.0, 0, 0], [0, 0.5, 0.5], preference_vectors]
#)
objective_values = Hx[0][0].ob

(
    survived_solutions,
    survived_sol_idx,
    optimal_solution_set,
    index_in_complete_solution_set,
    mapping,
) = PreferenceSurvival(preference_vectors).do(objective_values, n_survive=5)

In [34]:
mapping

[(array([0. , 0.5, 0.5]), array([0.10833333, 0.05230733, 0.05774504]), 495),
 (array([0.5, 0.5, 0. ]), array([0.105     , 0.06567062, 0.06509106]), 478),
 (array([0.5, 0. , 0.5]), array([0.10333333, 0.12146014, 0.06685083]), 415),
 (array([1., 0., 0.]), array([0.10333333, 0.12146014, 0.06685083]), 415),
 (array([0., 1., 0.]), array([0.11333333, 0.04565544, 0.07808471]), 489)]

In [35]:
optimal_solution_set

array([[0.10333333, 0.12146014, 0.06685083],
       [0.105     , 0.06567062, 0.06509106],
       [0.11333333, 0.04565544, 0.07808471],
       [0.10833333, 0.05230733, 0.05774504]])

In [36]:
index_in_complete_solution_set

array([415, 478, 489, 495])

In [37]:
survived_solutions

array([[0.12      , 0.27786128, 0.05749949],
       [0.24833333, 1.        , 0.        ],
       [0.105     , 0.06567062, 0.06509106],
       [0.10333333, 0.12146014, 0.06685083],
       [0.11333333, 0.04565544, 0.07808471]])

In [38]:
survived_sol_idx

array([131,   1, 478, 415, 489])

In [129]:
print(
    f"{optimal_solution_set.shape[0]}/{max_solutions} final solutions with respect to the {len(preference_vectors)} reference vector"
)

NameError: name 'max_solutions' is not defined

In [ ]:
preference_vectors

In [ ]:
optimal_solution_set

In [ ]:
np.unique(optimal_solution_set, axis=0)